<a href="https://colab.research.google.com/github/g4br/cfsAnlBacias/blob/main/Tutorial_de_GrADS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Aqui é instalado o GrADS para Ubuntu 18.04 com o comando apt
É instalado o pacote cartopy (linhas de divisão entre paises)  e é feito uma instalação do pacote shapely a força para corrigir um bug no cartopy.


In [ ]:
!apt install grads
!pip install cartopy
!pip install --no-binary shapely shapely --force


O comando git faz uma copia do programa py3grads do github



In [ ]:
!git clone https://github.com/meridionaljet/py3grads

Vá no diretorio py3grads

In [ ]:
%cd py3grads


Compila e instala o programa py3grads


In [ ]:
!python setup.py install

Volta a diretorio de trabalho (/content) 

In [ ]:
%cd ..

Use o programa wget para pegar o diretorio example.tar.gz no site do GrADS

In [ ]:
!wget ftp://cola.gmu.edu/grads/example.tar.gz

Descomprime o diretorio example.tar.gz

In [ ]:
!tar zxvf example.tar.gz

Notar que:
./model.ctl = contem o aquivo controle .ctl
./model.dat = contem o aquivo de dados na forma binaria 
./tutorial = contem o aquivo com os comando usado no tutorial do GrADS

Agora use o pacote py3grads para emular o GrADS no %%python
A vantagem de usar o py3grads em vez do python puro é levar ao %%python
100% de aquivos conhecidos na meterologia

In [ ]:
from py3grads import Grads 
ga = Grads(verbose=False) 
ga('open model.ctl') # abrindo o arquivo controle
ga('q file 1') # mostrar as variáveis no arquivo controle

In [ ]:
ga('set lon 0 360') #   sets longitude to 0 to 360  degrees West
ga('set lat -90 90') #     sets latitude -90 to 90 degrees North
ga('set t 1') # set tempo igual a 1
ps=ga.exp('ps') #pegue a variavel ps = Surface Pressure
lons = ga.exp('lon') # #pegue longitude
lats = ga.exp('lat') # #pegue latitude
[ps.min(),ps.max()] #mostre o valor menor e maior de ps

Agora use o cartopy e o matplotlib para plotar a variavel ps

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
plt.figure(figsize=(16,12)) # tamanho da figura
ax=plt.axes(projection=ccrs.PlateCarree()) # set a projeção
# plota em contornos [400,..104] com base no mapa de cores 'rainbow'
c1=plt.contourf(lons, lats, ps,[400,500,600,700,800,900,950,970,1000,1010,1020,1030,1040],cmap='rainbow')
# plota sobre o mapa acima com linhas com base no mapa de cores 'gray' para destacar
c2=plt.contour(lons, lats, ps,[400,500,600,700,800,900,950,970,1000,1010,1020,1030,1040],cmap='gray',linewidths=1)
#plot o mapa das linhas de costa.
ax.coastlines(color='k')
# plota a barra de cores
plt.colorbar(c1,orientation='horizontal')
#salva em arquivo
plt.savefig('pressao_em_superficie.png')

In [ ]:
ga('set lon -90') #   sets longitude to 90 degrees West
ga('set lat 40') #     sets latitude to 40 degrees North
ga('set lev 500')#     sets level to 500 mb
ga('set t 1')   #      sets time to first time step
hgt=ga.exp('hgt')  #         take a variable called 'hgt'

Plote em uma dimensão

In [ ]:
ga('set lon  -180 0') #   sets longitude to 90 degrees West
hgt=ga.exp('hgt')  #         
lons = ga.exp('lon')
plt.figure(figsize=(10,8))
plt.plot(lons,hgt)

Plote em duas dimensoes

In [ ]:

%matplotlib inline
ga('set lat 0 90')
ga('set lon 0 360')
ga('set lev 500')
lons = ga.exp('lon')
lats = ga.exp('lat')
hgt=ga.exp('hgt')  #  
from matplotlib import pyplot as plt
plt.figure(figsize=(10,8))
ax=plt.axes(projection=ccrs.PlateCarree()) # set a projeção
#plot o mapa das linhas de costa.
ax.coastlines(color='k')
plt.contourf(lons, lats, hgt)


Plote uma animação do geopotencial em 500 hPa e salve em aquivos separados

In [ ]:
for t in range(5):
  cmd='set t '+str(t+1) # Aqui é feita uma concatenação de um string e uma variavel inteira 
  ga(cmd) # e correponde ao 'set t ' t
  hgt=ga.exp('hgt') 
  plt.figure(figsize=(10,8))
  ax=plt.axes(projection=ccrs.PlateCarree()) # set a projeção
  #plot o mapa das linhas de costa. 
  ax.coastlines(color='k')
  plt.contourf(lons, lats,hgt ,cmap='RdGy',linewidths=2)
  
  plt.title('Altura do geopotencial (m)  em 500 hPa no tempo='+str(t))
  file='altura_do_geoptencial_500_t'+str(t)+'.png'
  plt.savefig(file)  

In [ ]:
!apt install imagemagick

In [ ]:
!convert  -delay 20 altura*.png movie.gif

Faça o mesmo para o vento em 200 hPa 

In [ ]:
ga('set lev 200')
for t in range(5):
  cmd='set t '+str(t+1) # Aqui é feita uma concatenação de um string e uma variavel inteira 
  ga(cmd) # e correponde ao 'set t ' t
  hgt=ga.exp('mag(u,v)') 
  plt.figure(figsize=(10,8))
  ax=plt.axes(projection=ccrs.PlateCarree()) # set a projeção
  #plot o mapa das linhas de costa. 
  ax.coastlines(color='k')
  plt.contourf(lons, lats,hgt ,cmap='jet',linewidths=2)
  plt.title('Altura do geopotencial (m)  em 500 hPa no tempo='+str(t))
  file='vento_em_200_t'+str(t)+'.png'
  plt.savefig(file)  

In [ ]:
!convert  -delay 20 altura*.png v200.gif